In [ ]:
import google.generativeai as genai
from google.colab import userdata

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

In [ ]:
gemini_lite = genai.GenerativeModel('gemini-2.0-flash-lite')
gemini_think = genai.GenerativeModel('gemini-2.5-pro')

In [ ]:
import json, re

def optimize_prompt(prompt, output):
    instruction = f'''
    You are a prompt-optimizer. Improve the prompt based on the model output.
    Return ONLY a JSON object with fields:
    - optimized_prompt
    - feedback
    - rationale

    Prompt: "{prompt}"
    Output: "{output}"
    '''

    response = gemini_think.generate_content(instruction)
    raw = response.text

    m = re.search(r"```json\n([\s\S]*?)\n```", raw)
    if m:
        data = json.loads(m.group(1))
    else:
        data = json.loads(raw)

    return data

In [ ]:
def run_optimization(prompt, iterations=5):
    current = prompt
    for i in range(1, iterations + 1):
        print(f"Iteracija: {i}")
        print("Prompt:", current)

        lite_output = gemini_lite.generate_content(current).text
        print("Lite Output:", lite_output)

        result = optimize_prompt(current, lite_output)
        print("Feedback:", result['feedback'])
        print("Rationale:", result['rationale'])
        print("Optimized Prompt:", result['optimized_prompt'])
        print()

        current = result['optimized_prompt']

    return current

In [ ]:
initial_prompt = "Objasni u detalje pocetniku kako funkcionira motor auta."
run_optimization(initial_prompt, 3)